In [1]:
!hostname

c303-006.ls6.tacc.utexas.edu


This notebook is for debugging only. Actual functionalliyt is in the bash script.

In [2]:
import dask

from xarray.groupers import TimeResampler

from rainproj.era_arco import ERA
from rainproj.rainreader2 import RainReader2
import os

WORK = os.environ.get('WORK')
SCRATCH = os.environ.get('SCRATCH')

In [3]:
from pathlib import Path

In [ ]:
# From htop, roughly estimated that 20 task will consume 80 GB, note that TACC nodes have 251 GB, probably safe to use 40 workers avoid memory overload. Or maybe even stick to 20 since this is an IO bound task. Cpus will wait until all data is got,.
from rainproj.era_arco import run_download2

run_download2()

***

In [4]:

# {'time': TimeResampler("MS")}
ds_daily = ERA.load_ar_total_precipitation().sortby('latitude').rename({'latitude':'lat','longitude':'lon'}).pipe(RainReader2.slice_rain).chunk(time=TimeResampler("MS")).pipe(RainReader2.resample_daily)
month_year_groups = ds_daily.groupby(time=TimeResampler("MS"))

def process_month(month_year_ds):
    df = month_year_ds.stack(idx=("time","lat","lon")).to_pandas().to_frame()
    month_year = df.index.get_level_values('time')[0].strftime('%Y-%m')
    df.to_parquet(f'{SCRATCH}/hindcast/era2/{month_year}.parquet')

# tasks = [dask.delayed(process_month)(month_year_ds) for month_year_ds in month_year_groups]

# dask.compute(tasks)


In [5]:
for time,mo_yr in month_year_groups:
    time = time
    break


In [13]:
from datetime import datetime
import pandas as pd
pd.Timestamp(time).strftime('%Y-%m')

'1978-12'

In [7]:
ERA.load_ar_total_precipitation().sortby('latitude').rename({'latitude':'lat','longitude':'lon'}).pipe(RainReader2.slice_rain).pipe(RainReader2.resample_daily)

<xarray.DataArray 'total_precipitation' (time: 16804, lat: 99, lon: 233)> Size: 2GB
dask.array<stack, shape=(16804, 99, 233), dtype=float32, chunksize=(1, 99, 233), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 396B 25.0 25.25 25.5 25.75 ... 48.75 49.0 49.25 49.5
  * lon      (lon) float32 932B 235.0 235.2 235.5 235.8 ... 292.5 292.8 293.0
  * time     (time) datetime64[ns] 134kB 1978-12-31 1979-01-01 ... 2025-01-01
Attributes:
    long_name:   24-Hour Total Precipitation
    short_name:  tp
    units:       m

In [8]:
ERA.load_ar_total_precipitation().sortby('latitude').rename({'latitude':'lat','longitude':'lon'}).pipe(RainReader2.slice_rain).chunk(time=TimeResampler("MS")).pipe(RainReader2.resample_daily)

<xarray.DataArray 'total_precipitation' (time: 16804, lat: 99, lon: 233)> Size: 2GB
dask.array<stack, shape=(16804, 99, 233), dtype=float32, chunksize=(1, 99, 233), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 396B 25.0 25.25 25.5 25.75 ... 48.75 49.0 49.25 49.5
  * lon      (lon) float32 932B 235.0 235.2 235.5 235.8 ... 292.5 292.8 293.0
  * time     (time) datetime64[ns] 134kB 1978-12-31 1979-01-01 ... 2025-01-01
Attributes:
    long_name:   24-Hour Total Precipitation
    short_name:  tp
    units:       m

In [ ]:
dask.compute(tasks)

In [32]:
import numpy as np
# https://docs.xarray.dev/en/stable/user-guide/groupby.html

process_month(month_year_groups[np.datetime64('1979-12-01T00:00:00.000000000')])

In [24]:
test.index.get_level_values('time')[0].strftime('%Y-%m')

'1979-12'

In [ ]:
# Chunking can be useful
# ds_daily = ds.pipe(RainReader2.resample_daily).chunk(time=TimeResampler("MS"))

In [ ]:
# import pandas as pd
# time =ds_daily.time[0]
# date = str(time.values).split('T')[0]
# tt = pd.concat({time.values:ds_daily.sel(time=time).stack(idx=("lat","lon")).to_series()},names=['date'])
# tt

date        lat   lon   
1978-12-31  25.0  235.00    0.000123
                  235.25    0.000085
                  235.50    0.000116
                  235.75    0.000162
                  236.00    0.000189
                              ...   
            49.5  292.00    0.001550
                  292.25    0.001651
                  292.50    0.001742
                  292.75    0.001430
                  293.00    0.001056
Name: total_precipitation, Length: 23067, dtype: float32

In [ ]:
# print(ds.time[0])

<xarray.DataArray 'time' ()> Size: 8B
array('1900-01-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 8B 1900-01-01


In [55]:
from joblib import Parallel, delayed
import os

def run(ds,time):
    filename_date = str(time.values).split('T')[0]
    stacked = pd.concat({time.values:ds.sel(time=time).stack(idx=("lat","lon")).to_series()},names=['date']).to_frame()
    stacked.to_parquet(f'{SCRATCH}/hindcast/era/{filename_date}.parquet')
    # return stacked
    
parallel = Parallel(n_jobs=os.cpu_count()-1)
silent = parallel(delayed(run)(ds_daily,time) for time in ds_daily.time)

# [for i,_ in enumerate(ds_daily.time)]

: 

In [33]:
ds_daily.sel(time=ds_daily.time[0]).stack(idx=("lat","lon")).to_series()

lat   lon   
25.0  235.00    0.000123
      235.25    0.000085
      235.50    0.000116
      235.75    0.000162
      236.00    0.000189
                  ...   
49.5  292.00    0.001550
      292.25    0.001651
      292.50    0.001742
      292.75    0.001430
      293.00    0.001056
Name: total_precipitation, Length: 23067, dtype: float32

In [32]:
ds_stack = ds_daily.stack(idx=("lat","lon",)).to_series()

In [11]:
ds_stack

<xarray.DataArray 'total_precipitation' (idx: 387617868)> Size: 2GB
dask.array<reshape, shape=(387617868,), dtype=float32, chunksize=(33608,), chunktype=numpy.ndarray>
Coordinates:
  * idx      (idx) object 3GB MultiIndex
  * lat      (idx) float32 2GB 25.0 25.0 25.0 25.0 25.0 ... 49.5 49.5 49.5 49.5
  * lon      (idx) float32 2GB 235.0 235.0 235.0 235.0 ... 293.0 293.0 293.0
  * time     (idx) datetime64[ns] 3GB 1978-12-31 1979-01-01 ... 2025-01-01
Attributes:
    long_name:   24-Hour Total Precipitation
    short_name:  tp
    units:       m

***

In [2]:
ds_subset = ds.sel(latitude=slice(49.5904,24.9493),longitude=slice(360-125.0011,360-66.9326),time=slice('1978-12-31','2025-01-01'))

In [ ]:
ds.sel(latitude=slice(49.5904,24.9493),longitude=slice(360-125.0011,360-66.9326),time=slice('1978-12-31','2025-01-01'))

NameError: name 'ds' is not defined

In [3]:
ds_subset

<xarray.DataArray 'total_precipitation' (time: 403296, latitude: 99,
                                         longitude: 233)> Size: 37GB
dask.array<getitem, shape=(403296, 99, 233), dtype=float32, chunksize=(48, 99, 233), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 396B 49.5 49.25 49.0 48.75 ... 25.5 25.25 25.0
  * longitude  (longitude) float32 932B 235.0 235.2 235.5 ... 292.5 292.8 293.0
  * time       (time) datetime64[ns] 3MB 1978-12-31 ... 2025-01-01T23:00:00
Attributes:
    long_name:   Total precipitation
    short_name:  tp
    units:       m

In [4]:
ds_daily = ds_subset.resample(time='1D').sum()

In [6]:
stacked = ds_daily.stack(idx=("latitude","longitude","time")).to_series().compute()

In [47]:
ds_daily.to_netcdf('data/era5_total_precipitation.nc')

In [45]:
import zarr
from numcodecs.zarr3 import Blosc

encoding = {
    "total_precipitation": {
        "compressor": Blosc(
            cname="zstd",
            clevel=6,
        ),
    }
}
for coord in ds_daily.coords:
    encoding[coord] = {"compressors": None}
ds_daily.to_zarr('data/era5_total_precipitation.zarr',mode='w',encoding=encoding)

In [ ]:
# https://docs.mapbox.com/mapbox-tiling-service/examples/bbox/ [ -125.0011, 24.9493, -66.9326, 49.5904 ][ minlon, minlat, maxlon, maxlat ]

In [ ]:
#.resample(time='1D').sum()